#### 1.

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:

    - взять в тренировочный набор турниры с dateStart из 2019 года; 
    - в тестовый — турниры с dateStart из 2020 года.

In [1]:
import os
import pickle

import numpy as np
import pandas as pd
from scipy.special import logit, expit
from sklearn.linear_model import LogisticRegression, Ridge

from collections import defaultdict, Counter
from scipy.sparse import coo_matrix

In [2]:
with open("results.pkl", "rb") as f:
    results = pickle.load(f)
with open("tournaments.pkl", "rb") as f:
    tournaments = pickle.load(f)

Выбираем игры от 2019 года с информацией по ответам и игрокам

In [3]:
results_new = defaultdict(list)
tournaments_new = {}

for id, value in results.items():
    y = int(tournaments[id]["dateStart"][0:4])
    if y in [2019, 2020]:
        for team in value:
            mask = team.get("mask", None)
            team_members = team.get("teamMembers", [])
            if mask and team_members:
                results_new[id].append(team)
    else:
        continue

tournaments_new = {k: tournaments[k] for k, v in results_new.items()}
results, tournaments = dict(results_new), tournaments_new

In [4]:
len(results), len(tournaments)

(848, 848)

Делим данные на трейн и тест

In [5]:
train_data, test_data = {}, {}
for id, tournament in results.items():
    y = int(tournaments[id]["dateStart"][0:4])
    
    if y == 2019:
        train_data[id] = {"tournament_name": tournaments[id]["name"]}
        team_results = []
        for team_result in tournament:
            team_info = {"team_id": team_result["team"]["id"], 
                         "mask": team_result["mask"], 
                         "position": team_result["position"],
                         "teamMembers": [team_member["player"]["id"] for team_member in team_result['teamMembers']]}
            team_results.append(team_info)
        train_data[id]["tournament_result"] = team_results
  
    elif y == 2020:
        test_data[id] = {"tournament_name": tournaments[id]["name"]}
        team_results = []
        for team_result in tournament:
            team_info = {"team_id": team_result["team"]["id"], 
                         "mask": team_result["mask"], 
                         "position": team_result["position"],
                         "teamMembers": [team_member["player"]["id"] for team_member in team_result['teamMembers']]}
            team_results.append(team_info)
        test_data[id]["tournament_result"] = team_results

In [9]:
len(train_data), len(test_data)

(675, 173)

Заменяем id игроков, отсутствующих в тесте, и игроков, у которых мало игр

In [10]:
games_by_player = Counter()
for tournament in train_data.values():
    for team in tournament["tournament_result"]:
        games_by_player += Counter({member: 1 for member in team["teamMembers"]})

low_exp_players = set()
for member, games in games_by_player.most_common():
    if games <= 5:
        low_exp_players.add(member)

y2019_players = set()
for tournament in train_data.values():
    for team in tournament["tournament_result"]:
        team["teamMembers"] = set([-1 if member in low_exp_players else member for member in team["teamMembers"]])
        y2019_players.update(team["teamMembers"])
        
for tournament in test_data.values():
    for team in tournament["tournament_result"]:
        team["teamMembers"] = set([-1 if member not in y2019_players else member for member in team["teamMembers"]])

In [11]:
len(train_data), len(test_data)

(675, 173)

In [12]:
members_and_questions = set()
for id, tournament in train_data.items():
    for team in tournament["tournament_result"]:
        members_and_questions.update(team["teamMembers"])
        questions_ids = (f"{id}_{question_num}" for question_num in range(len(team["mask"])))
        members_and_questions.update(questions_ids)
members_and_questions = {v: i for i, v in enumerate(members_and_questions)}

In [13]:
def prepare_data(data, train):
    global members_and_questions
    rows = []
    cols = []
    y = []
    current_row = 0
    for id, tournament in data.items():
        for team in tournament["tournament_result"]:
            for quest_numb, mask in enumerate(team["mask"]):
                try:
                    y.extend([int(mask)] * len(team["teamMembers"]))
                except ValueError:
                    continue
                for member in team["teamMembers"]:
                    rows.append(current_row)
                    cols.append(members_and_questions[member])
                    if train:    
                        rows.append(current_row)
                        cols.append(members_and_questions[f"{id}_{quest_numb}"])
                    current_row += 1
                
    rows = np.asarray(rows, dtype=np.int32)
    cols = np.asarray(cols, dtype=np.int32)
    data = np.ones(len(rows))
    y = np.asarray(y, dtype=np.int8)
        
    X = coo_matrix((data, (rows, cols)), shape=(len(y), len(members_and_questions)))
    return X, y

X_train, y_train = prepare_data(train_data, True)
X_test, y_test = prepare_data(test_data, False)

In [14]:
X_train.shape, len(y_train), X_test.shape, len(y_test)

((18000070, 49836), 18000070, (3795338, 49836), 3795338)

#### 2.

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:

    - повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
    - в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
    - для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


С помощью модели логистической регресии будем предсказывать вероятность правильного ответа для отдельного игрока на конкретный вопрос, при этом на основе данных 2019 года считаем, что если команда ответила на вопрос верно, то и все игроки ответили верно на тот же вопрос

In [15]:
model = LogisticRegression()
model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [16]:
from sklearn.metrics import log_loss

predict_train = model.predict_proba(X_train)[:, 1]
predict_test = model.predict_proba(X_test)[:, 1]

print (log_loss(y_train, predict_train))
print (log_loss(y_test, predict_test))
print (len(predict_train), len(predict_test))

0.5061998186976242
0.6987877226649079


(18000070, 3795338)

#### 3.

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:

    - предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
    - в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Для построения рейтинг-системы необходимо отсортировать команды в зависимости от того, как много игроков в них правильно отвечают на вопросы. Будем считать, что команда дает верный ответ при верном ответе хотя бы одного игрока.
Готовим датафрейм с местами команд для тестовых данных.

In [17]:
tournament_ids = []
team_ids = []
positions = []
for id, tournament in test_data.items():
    for team in tournament["tournament_result"]:
        tournament_ids.append(id)
        team_ids.append(team["team_id"])
        positions.append(team["position"])

true_positions = pd.DataFrame.from_dict({
    'tournament_id': tournament_ids,
    'team_id': team_ids,
    'position_true': positions,
})

Готовим массивы с id игр и id команд с тем же размером, что и у предсказаний выше

In [18]:
is_answered = []
tournament_ids_test = []
team_ids_test = []
for id, tournament in test_data.items():
    for team in tournament["tournament_result"]:
        team_id = team["team_id"]
        for answer in team["mask"]:
            try:
                is_answered.extend([int(answer)] * len(team["teamMembers"]))
                tournament_ids_test.extend([id] * len(team["teamMembers"]))
                team_ids_test.extend([team_id] * len(team["teamMembers"]))
            except ValueError:
                continue

tournament_ids_test = np.asarray(tournament_ids_test, dtype=np.int32)
team_ids_test = np.asarray(team_ids_test, dtype=np.int32)

Готовим датафрейм с предсказанными местами команд для тестовых данных

In [19]:
predicted_positions = pd.DataFrame.from_dict({
    'tournament_id': tournament_ids_test,
    'team_id': team_ids_test,
    '1-predict': 1 - predict_test,
})
    
predicted_positions = predicted_positions.groupby(["tournament_id", "team_id"]).agg("prod").reset_index()
predicted_positions["position_pred"] = predicted_positions.groupby("tournament_id")["1-predict"].rank("dense")

In [20]:
from scipy.stats import kendalltau, spearmanr

merge = pd.merge(predicted_positions, true_positions, on=["tournament_id", "team_id"])

kendall = []
spearman = []
tournam_ids = merge['tournament_id'].unique()

for tournam_id in tournam_ids:
    kendall += [kendalltau(merge[merge['tournament_id']==tournam_id]["position_pred"], merge[merge['tournament_id']==tournam_id]["position_true"]).correlation]
    spearman += [spearmanr(merge[merge['tournament_id']==tournam_id]["position_pred"], merge[merge['tournament_id']==tournam_id]["position_true"]).correlation] 

kendall = np.asarray(kendall)
spearman = np.asarray(spearman)
kendall[np.isnan(kendall)] = 0.0
spearman[np.isnan(spearman)] = 0.0

kendall_corr = np.mean(kendall)
spearman_corr = np.mean(spearman)
print ('Ранговые корреляции Спирмена и Кендалла равны', spearman_corr, 'и', kendall_corr)

Ранговые корреляции Спирмена и Кендалла равны 0.7512559356926288 и 0.599376085519689


#### 4.

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:

    - предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
    - разработайте EM-схему для обучения этой модели, реализуйте её в коде;
    - обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.

На Е шаге будем оценивать мат ожидание скрытой переменной Z для ответов игроков.
Z будет определять событие ответа игрока при условии ответа команды.
На М шаге оцениваем ответы игроков при имеющихся сложностях вопросов, опираясь на Z

Готовим массивы с id игр, id команд, id игроков с тем же размером, что и X_train
на каждого участника

In [21]:
is_answered = []
tournament_ids_train = []
team_ids_train = []
player_ids_train = []
questions_train = []
for id, tournament in train_data.items():
    for team in tournament["tournament_result"]:
        team_id = team["team_id"]
        for question, answer in enumerate(team["mask"]):
            try:
                is_answered.extend([int(answer)] * len(team["teamMembers"]))
                tournament_ids_train.extend([id] * len(team["teamMembers"]))
                team_ids_train.extend([team_id] * len(team["teamMembers"]))
                questions_train.extend([question] * len(team["teamMembers"]))
            except ValueError:
                continue
            for member in team["teamMembers"]:
                player_ids_train.append(member)

tournament_ids_train = np.asarray(tournament_ids_train, dtype=np.int32)
team_ids_train = np.asarray(team_ids_train, dtype=np.int32)
questions_train = np.asarray(questions_train, dtype=np.int32)

In [22]:
for s in range(1, 11):
    # E-шаг
  
    predicts_and_other_info = pd.DataFrame.from_dict({
        'tournament_id': tournament_ids_train,
        'team_id': team_ids_train,
        'player_id': player_ids_train,
        'questions': questions_train,
        'predict': predict_train,
      })
    predicts_and_other_info["1-predict"] = 1 - predicts_and_other_info["predict"]
  
    predicts_by_teams = predicts_and_other_info.drop(columns=["player_id", "predict"]).groupby(["tournament_id", "team_id", "questions"]).agg("prod").reset_index()
    predicts_by_teams["team_predict"] = 1 - predicts_by_teams["1-predict"]
  
    predicts_and_other_info = pd.merge(predicts_and_other_info.drop(columns="1-predict"), predicts_by_teams.drop(columns="1-predict"), on=["tournament_id", "team_id", "questions"])
    predicts_and_other_info["z"] = predicts_and_other_info["predict"] / predicts_and_other_info["team_predict"]
  
    z = predicts_and_other_info["z"].values
    z = np.where(y_train == 0, 0, z)
    z = np.clip(z, 1e-6, 1 - 1e-6)
    
    # M-шаг
  
    model = Ridge(alpha=5, solver="auto", tol=0.0001)
    model.fit(X_train, logit(z))
    predict_train = expit(model.predict(X_train))
    predict_test = expit(model.predict(X_test))
    
    predicted_positions = pd.DataFrame.from_dict({
        'tournament_id': tournament_ids_test,
        'team_id': team_ids_test,
        '1-predict': 1 - predict_test,
        })
        
    predicted_positions = predicted_positions.groupby(["tournament_id", "team_id"]).agg("prod").reset_index()
    predicted_positions["position_pred"] = predicted_positions.groupby("tournament_id")["1-predict"].rank("dense")
  
    merge = pd.merge(predicted_positions, true_positions, on=["tournament_id", "team_id"])
  
    kendall = []
    spearman = []
    tournam_ids = merge['tournament_id'].unique()
  
    for tournam_id in tournam_ids:
        kendall += [kendalltau(merge[merge['tournament_id']==tournam_id]["position_pred"], merge[merge['tournament_id']==tournam_id]["position_true"]).correlation]
        spearman += [spearmanr(merge[merge['tournament_id']==tournam_id]["position_pred"], merge[merge['tournament_id']==tournam_id]["position_true"]).correlation] 
  
    kendall = np.asarray(kendall)
    spearman = np.asarray(spearman)
    kendall[np.isnan(kendall)] = 0.0
    spearman[np.isnan(spearman)] = 0.0
  
    kendall_corr = np.mean(kendall)
    spearman_corr = np.mean(spearman)
  
    print(f"На {s} шаге корреляция Кендалла - {kendall_corr}, корреляция Спирмена: {spearman_corr}")

На 1 шаге корреляция Кендалла - 0.6168274926391729, корреляция Спирмена: 0.7690108709847878
На 2 шаге корреляция Кендалла - 0.620477918470916, корреляция Спирмена: 0.7721958280011267
На 3 шаге корреляция Кендалла - 0.6228992944310386, корреляция Спирмена: 0.7743655065648926
На 4 шаге корреляция Кендалла - 0.6243231856102744, корреляция Спирмена: 0.7753059254924446
На 5 шаге корреляция Кендалла - 0.6247936899433085, корреляция Спирмена: 0.7753663756837581
На 6 шаге корреляция Кендалла - 0.6234096493169606, корреляция Спирмена: 0.7746554825746125
На 7 шаге корреляция Кендалла - 0.6233389986277601, корреляция Спирмена: 0.7745717327084001
На 8 шаге корреляция Кендалла - 0.6232920656827367, корреляция Спирмена: 0.7745412022353775
На 9 шаге корреляция Кендалла - 0.6234577045638449, корреляция Спирмена: 0.7746234400719666
На 10 шаге корреляция Кендалла - 0.6232816406473747, корреляция Спирмена: 0.7745259957803728


#### 5.

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [23]:
tournaments_weights = set()
for id, tournament in train_data.items():
    for team in tournament["tournament_result"]:
        questions_ids = (f"{id}_{question_num}" for question_num in range(len(team["mask"])))
        tournaments_weights.update(questions_ids)
tournaments_weights = {i: int(v.split("_")[0]) for i, v in enumerate(tournaments_weights) if isinstance(v, str)}

Отсортируем игры по сложности вопросов.
Сложность вопросов будем находить через отношение суммы весов модели на количество вопросов для каждого турнира

In [24]:
import heapq
from urllib.parse import quote

tournaments_level = defaultdict(lambda: [0, 0, 0]) # сумма весов, число вопросов, сложность вопросов

for weight_index, weight in enumerate(model.coef_):
    try:
        tournament_id = tournaments_weights[weight_index]
    except KeyError:
        continue
    tournaments_level[tournament_id][0] += weight
    tournaments_level[tournament_id][1] += 1

for _, difficulty in tournaments_level.items():
    difficulty[2] = difficulty[0] / difficulty[1]

hard_tournaments = [
    train_data[key]['tournament_name'] for key, value in tournaments_level.items()
    if -value[2] in heapq.nlargest(10, [-diff[2] for diff in tournaments_level.values()])
]
easy_tournaments = [
    train_data[key]['tournament_name'] for key, value in tournaments_level.items()
    if value[2] in heapq.nlargest(10, [diff[2] for diff in tournaments_level.values()])
]

In [25]:
print('Турниры со сложными вопросами:')
for t in hard_tournaments:
    print (t, 'https://db.chgk.info/search/questions/' + quote(t))

Турниры со сложными вопросами:
Синхрон-lite. Выпуск XXII https://db.chgk.info/search/questions/%D0%A1%D0%B8%D0%BD%D1%85%D1%80%D0%BE%D0%BD-lite.%20%D0%92%D1%8B%D0%BF%D1%83%D1%81%D0%BA%20XXII
Синхрон с человеческим лицом IV https://db.chgk.info/search/questions/%D0%A1%D0%B8%D0%BD%D1%85%D1%80%D0%BE%D0%BD%20%D1%81%20%D1%87%D0%B5%D0%BB%D0%BE%D0%B2%D0%B5%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%BC%20%D0%BB%D0%B8%D1%86%D0%BE%D0%BC%20IV
Уходящая натура https://db.chgk.info/search/questions/%D0%A3%D1%85%D0%BE%D0%B4%D1%8F%D1%89%D0%B0%D1%8F%20%D0%BD%D0%B0%D1%82%D1%83%D1%80%D0%B0
Умлаут speсial: Сова в пабе https://db.chgk.info/search/questions/%D0%A3%D0%BC%D0%BB%D0%B0%D1%83%D1%82%20spe%D1%81ial%3A%20%D0%A1%D0%BE%D0%B2%D0%B0%20%D0%B2%20%D0%BF%D0%B0%D0%B1%D0%B5
Первенство Сибири. Пролог https://db.chgk.info/search/questions/%D0%9F%D0%B5%D1%80%D0%B2%D0%B5%D0%BD%D1%81%D1%82%D0%B2%D0%BE%20%D0%A1%D0%B8%D0%B1%D0%B8%D1%80%D0%B8.%20%D0%9F%D1%80%D0%BE%D0%BB%D0%BE%D0%B3
Зеркало Кубка Мюнхгаузена https://db.chgk.inf

In [26]:
print('Турниры с простыми вопросами:')
for t in easy_tournaments:
    print (t, 'https://db.chgk.info/search/questions/' + quote(t))

Турниры с простыми вопросами:
Командум https://db.chgk.info/search/questions/%D0%9A%D0%BE%D0%BC%D0%B0%D0%BD%D0%B4%D1%83%D0%BC
Синхронный кубок Беларуси https://db.chgk.info/search/questions/%D0%A1%D0%B8%D0%BD%D1%85%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9%20%D0%BA%D1%83%D0%B1%D0%BE%D0%BA%20%D0%91%D0%B5%D0%BB%D0%B0%D1%80%D1%83%D1%81%D0%B8
KFC https://db.chgk.info/search/questions/KFC
Умами https://db.chgk.info/search/questions/%D0%A3%D0%BC%D0%B0%D0%BC%D0%B8
Чемпионат Минска. Лига Б. Тур первый https://db.chgk.info/search/questions/%D0%A7%D0%B5%D0%BC%D0%BF%D0%B8%D0%BE%D0%BD%D0%B0%D1%82%20%D0%9C%D0%B8%D0%BD%D1%81%D0%BA%D0%B0.%20%D0%9B%D0%B8%D0%B3%D0%B0%20%D0%91.%20%D0%A2%D1%83%D1%80%20%D0%BF%D0%B5%D1%80%D0%B2%D1%8B%D0%B9
Синхрон студенческого фестиваля Этажи https://db.chgk.info/search/questions/%D0%A1%D0%B8%D0%BD%D1%85%D1%80%D0%BE%D0%BD%20%D1%81%D1%82%D1%83%D0%B4%D0%B5%D0%BD%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B3%D0%BE%20%D1%84%D0%B5%D1%81%D1%82%D0%B8%D0%B2%D0%B0%D0%BB%D1%8F%20%D0%AD%D1%82%D0%B0

#### 6.

Бонус: постройте топ игроков по предсказанной вашей моделью силе игры, а рядом с именами игроков напишите общее число вопросов, которое они сыграли. Скорее всего, вы увидите, что топ занят игроками, которые сыграли совсем мало вопросов, около 100 или даже меньше; если вы поищете их в официальном рейтинге ЧГК, вы увидите, что это какие-то непонятные ноунеймы. В baseline-модели, скорее всего, такой эффект будет гораздо слабее.
Это естественное свойство модели: за счёт EM-схемы влияние 1-2 удачно сыгранных турниров будет только усиливаться, потому что неудачных турниров, чтобы его компенсировать, у этих игроков нет. Более того, это не мешает метрикам качества, потому что если эти игроки сыграли всего 1-2 турнира в 2019-м, скорее всего они ничего или очень мало сыграли и в 2020, и их рейтинги никак не влияют на качество тестовых предсказаний. Но для реального рейтинга такое свойство, конечно, было бы крайне нежелательным. Давайте попробуем его исправить:

    - сначала жёстко: выберите разумную отсечку по числу вопросов, учитывая, что в одном турнире их обычно 30-50;
    - можно ли просто выбросить игроков, которые мало играли, и переобучить модель? почему? предложите, как нужно изменить модель, чтобы не учитывать слишком мало сыгравших, и переобучите модель;
    - но всё-таки это не слишком хорошее решение: если выбрать маленькую отсечку, будут ноунеймы в топе, а если большую, то получится, что у нового игрока слишком долго не будет рейтинга; скорее всего, никакой “золотой середины” тут не получится;
    - предложите более концептуальное решение для топа игроков в рейтинг-листе; если получится, реализуйте его на практике (за это уж точно будут серьёзные бонусные баллы).

In [27]:
is_answered = []
player_ids_test = []
questions_test = []
for id, tournament in test_data.items():
    for team in tournament["tournament_result"]:
        for question, answer in enumerate(team["mask"]):
            try:
                is_answered.extend([int(answer)] * len(team["teamMembers"]))
                questions_test.extend([question] * len(team["teamMembers"]))
            except ValueError:
                continue
            for member in team["teamMembers"]:
                player_ids_test.append(member)

questions_test = np.asarray(questions_test, dtype=np.int32)

In [28]:
df_test_players = pd.DataFrame.from_dict({
  'tournament_id': tournament_ids_test,
  'team_id': team_ids_test,
  'player_id': player_ids_test,
  'questions': questions_test,
  'predict': predict_test,
})
df_test_players["1-predict"] = 1 - df_test_players["predict"]
df_test_players["predict10"] = df_test_players["predict"]*10

Перемножим значения предиктов для каждого игрока.
Значит игроки с более высокими вероятностями ответов и будут иметь высокие позиции.

In [29]:
df_test_players_ratio = df_test_players[["player_id", 'predict10']].groupby(["player_id"]).agg("prod").reset_index()
df_test_players_quest_numb = df_test_players[["player_id", 'questions']].groupby(["player_id"]).agg("count").reset_index()
df_test_players_games = df_test_players[["player_id", 'tournament_id']].drop_duplicates().reset_index()
df_test_players_games = df_test_players_games.drop(columns="index").groupby(["player_id"]).agg("count").reset_index()
merge = pd.merge(df_test_players_ratio, df_test_players_quest_numb, on=["player_id"])
merge = pd.merge(merge, df_test_players_games, on=["player_id"])
merge.columns = ['player_id', 'ratio', 'questions', 'tournaments']
merge = merge.sort_values(by = 'ratio', ascending=False)
merge

,player_id,ratio,questions,tournaments
548,7008,inf,1033,23
2421,30152,inf,1669,43
2192,27403,2.677945e+144,517,13
1595,19915,8.451044e+139,1047,25
1482,18332,6.570947e+126,1295,33
...,...,...,...,...
6462,116490,0.000000e+00,224,6
6463,116497,0.000000e+00,449,12
6465,116519,0.000000e+00,143,4
6466,116520,0.000000e+00,143,4


In [39]:
best_players = list(merge['player_id'][0:5].values)
best_members = {}
for id, tournament in results.items():
    y = int(tournaments[id]["dateStart"][0:4])
    if y in [2020]:
        for team_results in tournament:
            for member in team_results['teamMembers']:
                member_id = member['player']['id']
                if member_id in best_players:
                    best_members[best_players.index(member_id)+1] = [member['player']['name'] + ' ' + member['player']['surname'], member_id]

In [40]:
import requests

def get_real_pos(player_id):
    url = f"https://rating.chgk.info/api/players/{player_id}/rating/last"
    try:
        pos = requests.get(url).json()['rating_position']
    except:
        pos = 0
    return pos

За счет того, что в разделе подготовки данных исключали игроков с числом игр до 5 в топ попали игроки, отыгравшие много вопросов.
Реальные рейтинги этих игроков не на самых последних местах, модель справилась неплохо.

In [41]:
from sortedcontainers import SortedDict

best_members = SortedDict(best_members)
for k, v in best_members.items():
    print(f"{k}: {v[0]}, отыграно вопросов {merge[merge.player_id==v[1]].questions.values[0]}, отыграно турниров {merge[merge.player_id==v[1]].tournaments.values[0]},",
          f"реальный рейтинг в базе ЧГК {get_real_pos(v[1])}")

1: Алексей Гилёв, отыграно вопросов 1033, отыграно турниров 23, реальный рейтинг в базе ЧГК 30
2: Артём Сорожкин, отыграно вопросов 1669, отыграно турниров 43, реальный рейтинг в базе ЧГК 1
3: Максим Руссо, отыграно вопросов 517, отыграно турниров 13, реальный рейтинг в базе ЧГК 5
4: Александр Марков, отыграно вопросов 1047, отыграно турниров 25, реальный рейтинг в базе ЧГК 51
5: Александр Либер, отыграно вопросов 1295, отыграно турниров 33, реальный рейтинг в базе ЧГК 7


#### 7.

Бонус: игроки со временем учатся играть лучше (а иногда бывает и наоборот). А в нашей модели получается, что первые неудачные турниры новичка будут тянуть его рейтинг вниз всю жизнь — это нехорошо, рейтинг должен быть достаточно гибким и иметь возможность меняться даже у игроков, отыгравших сотни турниров. Давайте попробуем этого добиться:

    - если хватит вычислительных ресурсов, сначала сделайте baseline совсем без таких схем, обучив рейтинги на всех турнирах с повопросными результатами, а не только на турнирах 2019 года; улучшилось ли качество предсказаний на 2020?
    - одну схему со временем мы уже использовали: брали для обучения только последний год турниров; примерно так делают, например, в теннисной чемпионской гонке; у этой схемы есть свои преимущества, но есть и недостатки (например, достаточно мало играть год, чтобы полностью пропасть из рейтинга);
    - предложите варианты базовой модели или алгоритма её обучения, которые могли бы реализовать изменения рейтинга со временем; если получится, реализуйте их на практике, проверьте, улучшатся ли предсказания на 2020.

In [42]:
with open("results.pkl", "rb") as f:
    results = pickle.load(f)
with open("tournaments.pkl", "rb") as f:
    tournaments = pickle.load(f)
    
results_upd = defaultdict(list)
tournaments_upd = {}

for id, value in results.items():
    for team in value:
        mask = team.get("mask", None)
        team_members = team.get("teamMembers", [])
        if mask and team_members:
            results_upd[id].append(team)

tournaments_upd = {k: tournaments[k] for k, v in results_upd.items()}
results, tournaments = dict(results_upd), tournaments_upd

In [43]:
train_data, test_data = {}, {}
for id, tournament in results.items():
    y = int(tournaments[id]["dateStart"][0:4])
    
    if y < 2020 and y >= 2015:
        train_data[id] = {"tournament_name": tournaments[id]["name"]}
        team_results = []
        for team_result in tournament:
            team_info = {"team_id": team_result["team"]["id"], 
                         "mask": team_result["mask"], 
                         "position": team_result["position"],
                         "teamMembers": [team_member["player"]["id"] for team_member in team_result['teamMembers']]}
            team_results.append(team_info)
        train_data[id]["tournament_result"] = team_results
    
    elif y == 2020:
        test_data[id] = {"tournament_name": tournaments[id]["name"]}
        team_results = []
        for team_result in tournament:
            team_info = {"team_id": team_result["team"]["id"], 
                         "mask": team_result["mask"], 
                         "position": team_result["position"],
                         "teamMembers": [team_member["player"]["id"] for team_member in team_result['teamMembers']]}
            team_results.append(team_info)
        test_data[id]["tournament_result"] = team_results

In [45]:
import tqdm

games_by_player = Counter()
for tournament in tqdm.tqdm_notebook(train_data.values()):
    for team in tournament["tournament_result"]:
        games_by_player += Counter({member: 1 for member in team["teamMembers"]})

low_exp_players = set()
for member, games in tqdm.tqdm_notebook(games_by_player.most_common()):
    if games <= 5:
        low_exp_players.add(member)

y2019_players = set()
for tournament in tqdm.tqdm_notebook(train_data.values()):
    for team in tournament["tournament_result"]:
        team["teamMembers"] = set([-1 if member in low_exp_players else member for member in team["teamMembers"]])
        y2019_players.update(team["teamMembers"])
        
for tournament in tqdm.tqdm_notebook(test_data.values()):
    for team in tournament["tournament_result"]:
        team["teamMembers"] = set([-1 if member not in y2019_players else member for member in team["teamMembers"]])

<ipython-input-45-0148a9722454>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for tournament in tqdm.tqdm_notebook(train_data.values()):


<ipython-input-45-0148a9722454>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for member, games in tqdm.tqdm_notebook(games_by_player.most_common()):


<ipython-input-45-0148a9722454>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for tournament in tqdm.tqdm_notebook(train_data.values()):


<ipython-input-45-0148a9722454>:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for tournament in tqdm.tqdm_notebook(test_data.values()):


In [46]:
import gc

gc.collect()

60

In [47]:
members_and_questions = set()
for id, tournament in train_data.items():
    for team in tournament["tournament_result"]:
        members_and_questions.update(team["teamMembers"])
        questions_ids = (f"{id}_{question_num}" for question_num in range(len(team["mask"])))
        members_and_questions.update(questions_ids)
members_and_questions = {v: i for i, v in enumerate(members_and_questions)}

In [48]:
def prepare_data(data, train):
    global members_and_questions
    rows = []
    cols = []
    y = []
    current_row = 0
    for id, tournament in data.items():
        for team in tournament["tournament_result"]:
            for quest_numb, mask in enumerate(team["mask"]):
                try:
                    y.extend([int(mask)] * len(team["teamMembers"]))
                except ValueError:
                    continue
                for member in team["teamMembers"]:
                    rows.append(current_row)
                    cols.append(members_and_questions[member])
                    if train:    
                        rows.append(current_row)
                        cols.append(members_and_questions[f"{id}_{quest_numb}"])
                    current_row += 1
                
    rows = np.asarray(rows, dtype=np.int32)
    cols = np.asarray(cols, dtype=np.int32)
    data = np.ones(len(rows))
    y = np.asarray(y, dtype=np.int8)
        
    X = coo_matrix((data, (rows, cols)), shape=(len(y), len(members_and_questions)))
    return X, y

X_train, y_train = prepare_data(train_data, True)
X_test, y_test = prepare_data(test_data, False)

In [49]:
model = LogisticRegression()
model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [50]:
from sklearn.metrics import log_loss

predict_train = model.predict_proba(X_train)[:, 1]
predict_test = model.predict_proba(X_test)[:, 1]

print (log_loss(y_train, predict_train))
print (log_loss(y_test, predict_test))

0.5123904175997495
0.7177946312695239


Сформировать трейн за все время до 2020 года не вышло, так как не хватало памяти,
но при расширении с 2015 по 2019 года результаты улучшились на пару десятых для обычной модели

In [51]:
tournament_ids = []
team_ids = []
positions = []
for id, tournament in test_data.items():
    for team in tournament["tournament_result"]:
        tournament_ids.append(id)
        team_ids.append(team["team_id"])
        positions.append(team["position"])

true_positions = pd.DataFrame.from_dict({
    'tournament_id': tournament_ids,
    'team_id': team_ids,
    'position_true': positions,
})

In [52]:
is_answered = []
tournament_ids_test = []
team_ids_test = []
for id, tournament in test_data.items():
    for team in tournament["tournament_result"]:
        team_id = team["team_id"]
        for answer in team["mask"]:
            try:
                is_answered.extend([int(answer)] * len(team["teamMembers"]))
                tournament_ids_test.extend([id] * len(team["teamMembers"]))
                team_ids_test.extend([team_id] * len(team["teamMembers"]))
            except ValueError:
                continue

tournament_ids_test = np.asarray(tournament_ids_test, dtype=np.int32)
team_ids_test = np.asarray(team_ids_test, dtype=np.int32)

In [53]:
predicted_positions = pd.DataFrame.from_dict({
    'tournament_id': tournament_ids_test,
    'team_id': team_ids_test,
    '1-predict': 1 - predict_test,
})
    
predicted_positions = predicted_positions.groupby(["tournament_id", "team_id"]).agg("prod").reset_index()
predicted_positions["position_pred"] = predicted_positions.groupby("tournament_id")["1-predict"].rank("dense")

In [54]:
from scipy.stats import kendalltau, spearmanr

merge = pd.merge(predicted_positions, true_positions, on=["tournament_id", "team_id"])

kendall = []
spearman = []
tournam_ids = merge['tournament_id'].unique()

for tournam_id in tournam_ids:
    kendall += [kendalltau(merge[merge['tournament_id']==tournam_id]["position_pred"], merge[merge['tournament_id']==tournam_id]["position_true"]).correlation]
    spearman += [spearmanr(merge[merge['tournament_id']==tournam_id]["position_pred"], merge[merge['tournament_id']==tournam_id]["position_true"]).correlation] 

kendall = np.asarray(kendall)
spearman = np.asarray(spearman)
kendall[np.isnan(kendall)] = 0.0
spearman[np.isnan(spearman)] = 0.0

kendall_corr = np.mean(kendall)
spearman_corr = np.mean(spearman)
print ('Ранговые корреляции Спирмена и Кендалла равны', spearman_corr, 'и', kendall_corr)

Ранговые корреляции Спирмена и Кендалла равны 0.7605950640046922 и 0.6064447732744951
